In [1]:
import cv2
import time
import numpy as np
import HandTrackingModule as htm
import math
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [ ]:
wCam, hCam = 640, 480

cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0

detector = htm.HandDetector()

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
volumeRange = volume.GetVolumeRange()
minimumVolume = volumeRange[0]
maximumVolume = volumeRange[1]

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)  # Flipping is needed to create mirroring
    # Find the hands
    detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)
    if len(lmList) != 0:
        # Filter based on size

        # Find the distance btwn index and thumb

        # Converting length to volume

        # Reduce resolution to make smoother.

        # Check fingers which are up

        # if pinky is down then set volume

        # drawings

        # Frame rate

        x1, y1 = lmList[4][1], lmList[4][2]
        x2, y2 = lmList[8][1], lmList[8][2]
        cx, cy = (x1+x2)//2, (y1+y2)//2
        cv2.circle(img, (x1, y1), 8, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), 8, (255, 0, 255), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 2)
        cv2.circle(img, (cx, cy), 8, (255, 0, 255), cv2.FILLED)

        length = math.hypot(x2-x1, y2-y1)
        # Hand range 10 to 150
        # Volume range -65.25 to 0
        vol = np.interp(length, [10, 150], [minimumVolume, maximumVolume])
        volume.SetMasterVolumeLevel(vol, None)

    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime

    cv2.putText(img, f'FPS:{int(fps)}', (48, 70),
                cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 3)
    cv2.imshow("Image", img)
    cv2.waitKey(1)

In [2]:
wCam, hCam = 640, 480
# boundingBox=[]
cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0

detector = htm.HandDetector()

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
volumeRange = volume.GetVolumeRange()
minimumVolume = volumeRange[0]
maximumVolume = volumeRange[1]

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)  # Flipping is needed to create mirroring
    # Find the hands
    detector.findHands(img)
    lmList,boundingBox= detector.findPosition(img,handNo=0, draw=False)
    if len(lmList) != 0:
        # Filter based on size
        print(boundingBox)
        # Find the distance btwn index and thumb

        # Converting length to volume

        # Reduce resolution to make smoother.

        # Check fingers which are up

        # if pinky is down then set volume

        # drawings

        # Frame rate

        x1, y1 = lmList[4][1], lmList[4][2]
        x2, y2 = lmList[8][1], lmList[8][2]
        cx, cy = (x1+x2)//2, (y1+y2)//2
        cv2.circle(img, (x1, y1), 8, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), 8, (255, 0, 255), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 2)
        cv2.circle(img, (cx, cy), 8, (255, 0, 255), cv2.FILLED)

        length = math.hypot(x2-x1, y2-y1)
        # Hand range 10 to 150
        # Volume range -65.25 to 0
        vol = np.interp(length, [10, 150], [minimumVolume, maximumVolume])
        volume.SetMasterVolumeLevel(vol, None)

    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime

    cv2.putText(img, f'FPS:{int(fps)}', (48, 70),
                cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 3)
    cv2.imshow("Image", img)
    cv2.waitKey(1)

(498, 215, 624, 387)
(486, 205, 614, 379)
(473, 200, 599, 375)
(468, 200, 596, 377)
(466, 201, 594, 379)
(463, 201, 591, 378)
(461, 202, 589, 378)
(459, 201, 588, 378)
(459, 201, 588, 378)
(458, 201, 587, 378)
(460, 202, 588, 379)
(460, 202, 588, 380)
(462, 205, 592, 382)
(463, 207, 593, 382)
(463, 207, 595, 384)
(464, 208, 595, 383)
(464, 208, 594, 384)
(465, 209, 594, 384)
(465, 208, 594, 384)
(465, 212, 597, 388)
(467, 211, 596, 388)
(466, 212, 597, 388)
(465, 211, 597, 387)
(467, 213, 596, 387)
(467, 214, 596, 388)
(466, 216, 596, 389)
(468, 216, 596, 390)
(465, 215, 595, 388)
(467, 214, 596, 387)
(468, 214, 597, 387)
(468, 213, 598, 386)
(483, 211, 609, 384)
(484, 208, 615, 383)
(489, 208, 620, 386)
(492, 208, 624, 388)
(497, 207, 629, 387)
(497, 207, 629, 389)
(496, 206, 627, 387)
(496, 206, 624, 386)
(496, 206, 625, 386)
(495, 205, 623, 385)
(498, 203, 623, 381)
(500, 203, 624, 379)
(501, 205, 623, 380)
(501, 206, 625, 380)
(501, 206, 625, 379)
(501, 207, 626, 379)
(504, 208, 62

KeyboardInterrupt: 